*Laetitia BERNE* <br>*Gilles DECKNACHE* <br>*Romain JALBERT*

# Homework n°1

### Importation des modules nécessaires


In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

import os, sys
sys.path.insert(0,os.path.abspath(os.path.join(os.path.abspath(''),'../utilities')))
import optmodel_utilities as opt_utils   

### Création du modèle avec la relaxation linéaire

In [2]:
def Create_model():

    model = AbstractModel()
    
    # Paramètre : nombre de sommets dans le graphe
    model.n = Param()
    
    # Ensemble des sommets du graphe
    model.V = RangeSet(model.n)
    
    # Paramètre : le coût d'installation d'un relai dans une localisation donnée
    model.c = Param(model.V)
    
    # Ensemble des arcs présents dans le graphe
    model.E = Set(within = model.V*model.V)
    
    
    # Variables
    model.x = Var(model.V, within = NonNegativeReals) 
    
    
    # Fonction Objectif : minimisation de la somme des coûts
    def Objectif(model):
        return sum(model.c[i]*model.x[i] for i in model.V)
    
    model.cout_total = Objective(rule=Objectif, sense=minimize)
    
    
    # Contraintes
    def regle_Contrainte(model, i, j):
        return model.x[i] + model.x[j] >= 1
    
    model.contrainte = Constraint(model.E, rule=regle_Contrainte)
   
    
    return model

### Création d'une instance et résolution 

In [3]:
# Choix du solver
optsolver = opt_utils.create_solver()

# Création du modèle
model = Create_model()

# Chargement des données depuis un fichier (et création d'une instance)
instance = model.create_instance('MRP_10.dat')        # MRP_20 ; MRP_40 ; MRP_80 ; MRP_160

#This command allow to print the full structure of the instance/model
#instance.pprint()

#solving the problem
results = optsolver.solve(instance)


### Affichage des résultats de la relaxation linéaire du problème

In [4]:
if (results.solver.status == SolverStatus.ok) and \
    (results.solver.termination_condition == TerminationCondition.optimal):

    opt_utils.print_objective_value(instance)
    opt_utils.print_point_from_model(instance)
    
    # dictionnaire pour stocker les valeurs des variables
    xi_values = {}

    for var in instance.component_objects(Var, active=True):
        varobject = getattr(instance, str(var))
        for index in varobject:
            xi_values[index] = varobject[index].value
    
    n = instance.n.value
    ensemble_E = instance.E.data()
    
    
else :
    print("Some problem occurred. Solver terminated with condition ", results.solver.termination_condition)

OBJ:  cout_total  =  15.0
x [ 1 ] =  0.5
x [ 2 ] =  0.5
x [ 3 ] =  0.0
x [ 4 ] =  0.0
x [ 5 ] =  1.0
x [ 6 ] =  1.0
x [ 7 ] =  0.5
x [ 8 ] =  0.0
x [ 9 ] =  0.5
x [ 10 ] =  0.5


### Randomized rounding

In [5]:
# Création de l'ensemble C qui contiendra les sommets où on va installer une antenne
C = []

# Conversion de l'ensemble des arcs en une liste, que l'on va pouvoir modifier (c'était un tuple jusqu'à présent)
E = list(ensemble_E)    

print(xi_values)
print(E)

{1: 0.5, 2: 0.5, 3: 0.0, 4: 0.0, 5: 1.0, 6: 1.0, 7: 0.5, 8: 0.0, 9: 0.5, 10: 0.5}
[(1, 6), (1, 7), (1, 9), (1, 10), (2, 7), (2, 9), (2, 10), (3, 6), (5, 8), (6, 7), (6, 9), (6, 10), (7, 9), (9, 10)]


In [6]:
# E_new est un dictionnaire, qui a pour clé tous les arcs de E et pour valeur 'oui' si l'arc correspondant a au moins
# un de ses sommets dans C, 'non' sinon
E_new={}
for arc in E:
    E_new[arc]='non'
# nb_arc est un compteur qui compte combien d'arc restants de E_new ne sont pas reliés à un des sommets de C
nb_arc = len(E)


# Initialisation de C :
# On ajoute les sommets qui ont déjà la valeur 1 et on supprime les arcs correspondants
for i in range(1, n+1):
    
    if xi_values[i] == 1 :
        C.append(i)
                
        for indice_arc in range(len(E)) :
            if E[indice_arc][0] == i or E[indice_arc][1] == i :
                if E_new[E[indice_arc]]=='non':
                    E_new[E[indice_arc]]='oui'
                    nb_arc = nb_arc - 1
                
        
print(C)
print(E_new)
print(nb_arc)

[5, 6]
{(1, 6): 'oui', (1, 7): 'non', (1, 9): 'non', (1, 10): 'non', (2, 7): 'non', (2, 9): 'non', (2, 10): 'non', (3, 6): 'oui', (5, 8): 'oui', (6, 7): 'oui', (6, 9): 'oui', (6, 10): 'oui', (7, 9): 'non', (9, 10): 'non'}
8


*Pour information :*<br>
Solution du problème BINAIRE de MRP_10.dat : cout_total = 17, avec <br>
x [ 1 ] =  1.0 <br>
x [ 2 ] =  1.0 <br>
x [ 3 ] =  0.0 <br>
x [ 4 ] =  0.0<br>
x [ 5 ] =  1.0<br>
x [ 6 ] =  1.0<br>
x [ 7 ] =  0.0<br>
x [ 8 ] =  0.0<br>
x [ 9 ] =  1.0<br>
x [ 10 ] =  0.0